In [21]:
import pandas as pd
from transformers import AutoTokenizer
import torch
import numpy as np

In [2]:
#import data
training_df = pd.read_csv('data/train.csv')
testing_df = pd.read_csv('data/test.csv')
training_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [15]:
#tokenize the training data
text = training_df['full_text'].to_list()
encoded_input = tokenizer(text, padding = True, truncation=True, return_tensors = 'pt')

#build a vocabulary vector
tokens = encoded_input['input_ids']

global max_num
max_num = 0
for token in tokens:
    max_temp = int(torch.max(token))
    if max_temp > max_num:
        max_num = max_temp
    else:
        continue


#convert the sequencial token to vector token
train_token_list = []
for token in tokens:
    init_vector = np.zeros(max_num+1)
    for position_index in token:
        i = int(position_index)
        if i > max_num:
            continue
        else:
            #init_vector[i] = init_vector[i]+1
            init_vector[i] = 1
    train_token_list.append(init_vector)

In [5]:
#tokenize the testing data
text = testing_df['full_text'].to_list()
encoded_input = tokenizer(text, padding = True, truncation=True, return_tensors = 'pt')

tokens = encoded_input['input_ids']

#convert the sequencial token to vector token
test_token_list = []
for token in tokens:
    init_vector = np.zeros(max_num+1)
    for position_index in token:
        i = int(position_index)
        if i > max_num:
            continue
        else:
            #init_vector[i] = init_vector[i]+1
            init_vector[i] = init_vector[i]+1
    test_token_list.append(init_vector) 

In [6]:
#get all the vocabulary scores in the training samples
vocabulary_scores = training_df.vocabulary

In [11]:
np.array(train_token_list)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [134]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

X = np.array(train_token_list)[1:,:]
y = vocabulary_scores.to_numpy()[1:]*2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

clf = BernoulliNB()
clf.fit(X_train, y_train)

BernoulliNB()

In [138]:
# predict y
y_pred = clf.predict(X_test)

In [139]:
#calculate the correct rate
count = 0
total_num = len(y_pred)
for i in range(0,len(y_pred)):
    if y_test[i] == y_pred[i]:
        count += 1
print('The correct rate is', count/total_num)

The correct rate is 0.42071611253196933


In [144]:
cm = confusion_matrix(y_test, y_pred)
test_name_list = []
original_list = []
for score in set(y):
    test_name_list.append('Predicted as '+str(score))
    original_list.append('True Value is '+str(score))

In [147]:
df = pd.DataFrame(cm, columns = test_name_list, index = original_list)

In [148]:
df

,Predicted as 2.0,Predicted as 3.0,Predicted as 4.0,Predicted as 5.0,Predicted as 6.0,Predicted as 7.0,Predicted as 8.0,Predicted as 9.0,Predicted as 10.0
True Value is 2.0,0,0,0,0,1,0,0,0,0
True Value is 3.0,0,0,0,0,3,0,0,0,0
True Value is 4.0,0,0,0,0,24,2,0,0,0
True Value is 5.0,0,0,0,0,98,4,0,0,0
True Value is 6.0,0,0,0,0,276,26,0,0,0
True Value is 7.0,0,0,0,0,149,53,1,0,0
True Value is 8.0,0,0,0,0,84,38,0,0,0
True Value is 9.0,0,0,0,0,6,10,1,0,0
True Value is 10.0,0,0,0,0,1,3,2,0,0
